In [1]:
from keras.models import load_model
import numpy as np
import sys
import os
import pickle as pk
import pandas as pd
from os import listdir
from os.path import isfile, join

from keras.utils import to_categorical ,Sequence
from keras import losses, models, optimizers
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

from keras.layers import (Convolution1D, Dense, Dropout, GlobalAveragePooling1D, 
                          GlobalMaxPool1D, Input, MaxPool1D, concatenate)

from keras.layers import (Convolution2D, GlobalAveragePooling2D, BatchNormalization, Flatten,
                          GlobalMaxPool2D, MaxPool2D, concatenate, Activation)

from keras import backend as K                      


# calculate accuracy
from sklearn.metrics import accuracy_score

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
# fname = df['fname'].tolist()
# label = df['label'].tolist()
# map_dict = pk.load(open('data/map.pkl' , 'rb'))
# fname


# ensemble for public_csv

In [89]:
map_dict = pk.load(open('data/map.pkl' , 'rb'))
folder_path = 'result/public/'
csv_files = [join(folder_path, f) for f in listdir(folder_path) if isfile(join(folder_path, f))]
acc_df = pd.read_csv('result/public_acc.csv')
acc_df.columns = ['csv','acc','weight']
mean = acc_df['acc'].mean()
std = acc_df['acc'].std()
print(mean , std)

result = np.zeros((9400 , 41))

for p in csv_files:
    try:
        ratio = acc_df[acc_df['csv'] == p.split('/')[-1]]['acc'].values[0]
        weights = acc_df[acc_df['csv'] == p.split('/')[-1]]['weight'].values[0]
    except:
#         print(p)
        continue
#     ratio = ratio * weight
    print(p,weights)
    if ratio > (mean + std):
        ratio = (ratio*weights + mean + std)*(10/6)
    elif ratio > mean:
        ratio = (ratio*weights + mean)*(10/8)
    elif ratio > (mean - std):
        ratio = (ratio*weights + mean - std)*1
    else:
        ratio = (ratio*weights - std)*0.8
    df = pd.read_csv(p)
    label = df['label'].tolist()
    predict = [x.split() for x in label]
    
    # convert to label
    for i,d in enumerate(predict):
        a = []
        for label in d:
            a.append(map_dict[label])
            
        predict[i] = a
    
    
    # convert to one-hot
    one_hot = np.zeros((df.shape[0] , 41))
    for i , d in enumerate( predict):

        for top ,label in enumerate( d ) :
            # 0 -> 1-(0 * 0.2) = 1
            # 1 -> 1-(1* 0.2) = 0.8
            # 2 -> 1-(2 *0.2) = 0.6
            if top==0:
                weight = 1 * ratio
            elif top==1:
                weight=0.8 * ratio #0.5
            elif top==2:
                weight = 0.6 * ratio #0.33
            one_hot[i][label] = weight
    
    result += one_hot
fname = df['fname'].tolist()
print(mean+std)

0.9225 0.0017320508075689429
result/public/public_voting_0919.csv 2
result/public/final_ens_stage3.csv 1
result/public/LGD_lpANDstack_123.csv 4
result/public/LGD_lp_stack_123.csv 3
0.924232050807569


In [88]:
result[1600] , weights

(array([ 0.        ,  0.        , 11.99662872,  0.        ,  0.        ,
         0.        ,  0.        , 14.9957859 ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         8.99747154,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]), 3)

# Create ans.csv

In [90]:
output = []
reverse_dict = pk.load(open('data/map_reverse.pkl' , 'rb'))

for i , d in enumerate( result):

    top3 = d.argsort()[-3:][::-1]
    result = [reverse_dict[x] for x in top3]
    s = ' '.join(result)
    output.append(s)
    
df = pd.DataFrame(output , columns = ['label'])
df.insert(0,'fname' , fname)
df.to_csv('result/public_voting_final.csv', index=False)
print(df.head(10))

          fname                               label
0  00063640.wav    Shatter Keys_jangling Tambourine
1  0013a1db.wav      Flute Trumpet Violin_or_fiddle
2  002bb878.wav  Bass_drum Knock Gunshot_or_gunfire
3  002d392d.wav         Bass_drum Knock Double_bass
4  00326aa9.wav             Oboe Clarinet Bass_drum
5  0038a046.wav      Bass_drum Electric_piano Knock
6  003995fa.wav           Squeak Clarinet Telephone
7  005ae625.wav      Acoustic_guitar Cello Clarinet
8  007759c4.wav            Clarinet Flute Telephone
9  008afd93.wav               Saxophone Flute Cello


In [ ]:
          fname                               label
0  00063640.wav    Shatter Keys_jangling Tambourine
1  0013a1db.wav      Flute Trumpet Violin_or_fiddle
2  002bb878.wav  Bass_drum Knock Gunshot_or_gunfire
3  002d392d.wav         Bass_drum Knock Double_bass
4  00326aa9.wav             Oboe Clarinet Bass_drum
5  0038a046.wav      Bass_drum Electric_piano Knock
6  003995fa.wav           Squeak Clarinet Telephone
7  005ae625.wav      Acoustic_guitar Cello Clarinet
8  007759c4.wav            Clarinet Flute Telephone
9  008afd93.wav               Saxophone Flute Cello

# kernel ens
* reproduce kernel result

**Ensembling 1D Conv Predictions**

In [47]:
train = pd.read_csv("data/train_label.csv")
LABELS = list(train.label.unique())

In [49]:
pred_list = []
for i in range(10):
    pred_list.append(np.load("data/kernel/freesound-prediction-file/test_predictions_%d.npy"%i))
prediction = np.ones_like(pred_list[0])
for pred in pred_list:
    prediction = prediction*pred
prediction = prediction**(1./len(pred_list))
# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("result/1d_conv_ensembled_submission.csv", index=False)
test

**Ensembling 2D Conv Predictions**

In [51]:
pred_list = []
for i in range(10):
    pred_list.append(np.load("data/kernel/freesound-prediction-data-2d-conv-reduced-lr/test_predictions_%d.npy"%i))
prediction = np.ones_like(pred_list[0])
for pred in pred_list:
    prediction = prediction*pred
prediction = prediction**(1./len(pred_list))
# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("result/2d_conv_ensembled_submission.csv", index=False)
test

,fname,label
0,00063640.wav,Shatter Tambourine Keys_jangling
1,0013a1db.wav,Flute Violin_or_fiddle Oboe
2,002bb878.wav,Bass_drum Snare_drum Knock
3,002d392d.wav,Bass_drum Snare_drum Flute
4,00326aa9.wav,Oboe Clarinet Electric_piano
5,0038a046.wav,Bass_drum Gong Cello
6,003995fa.wav,Squeak Meow Microwave_oven
7,005ae625.wav,Electric_piano Saxophone Acoustic_guitar
8,007759c4.wav,Clarinet Telephone Flute
9,008afd93.wav,Saxophone Flute Clarinet


**Ensembling 1D Conv and 2D Conv Predictions**

In [52]:
pred_list = []
for i in range(10):
    pred_list.append(np.load("data/kernel/freesound-prediction-data-2d-conv-reduced-lr/test_predictions_%d.npy"%i))
for i in range(10):
    pred_list.append(np.load("data/kernel/freesound-prediction-file/test_predictions_%d.npy"%i))
prediction = np.ones_like(pred_list[0])
for pred in pred_list:
    prediction = prediction*pred
prediction = prediction**(1./len(pred_list))
# Make a submission file
top_3 = np.array(LABELS)[np.argsort(-prediction, axis=1)[:, :3]]
predicted_labels = [' '.join(list(x)) for x in top_3]
test = pd.read_csv('data/sample_submission.csv')
test['label'] = predicted_labels
test[['fname', 'label']].to_csv("result/1d_2d_ensembled_submission.csv", index=False)
test

,fname,label
0,00063640.wav,Shatter Trumpet Finger_snapping
1,0013a1db.wav,Flute Oboe Violin_or_fiddle
2,002bb878.wav,Bass_drum Snare_drum Squeak
3,002d392d.wav,Bass_drum Snare_drum Squeak
4,00326aa9.wav,Oboe Clarinet Hi-hat
5,0038a046.wav,Bass_drum Knock Gong
6,003995fa.wav,Squeak Meow Microwave_oven
7,005ae625.wav,Acoustic_guitar Saxophone Electric_piano
8,007759c4.wav,Clarinet Telephone Flute
9,008afd93.wav,Saxophone Flute Clarinet
